In [1]:
import os
import pandas as pd
from datetime import datetime

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/cs3244_notebook/

/content/drive/MyDrive/cs3244_notebook


In [4]:
tesla_old = pd.read_csv('missing_dates_from_here.csv')
missing_dates = pd.read_csv('tesla_google_missing_date_ascending.csv')

In [5]:
print(tesla_old.head(3))

                                           Headlines       Dates  \
0  Tesla IPO Raises $226.1M, Stock Surges 41 Percent  2010-06-29   
1  2010: Tesla IPO: Electric-car maker’s stock up...  2010-06-29   
2    Tesla Motors IPO a win, but road ahead is bumpy  2010-06-29   

                                        Descriptions MappedDates  
0  Tesla Motors raised $226.1 million dollars in ...  2010-06-29  
1  2010: Tesla IPO: Electric-car maker's stock up...  2010-06-29  
2  The Tesla Motors IPO sold out quickly, and sha...  2010-06-29  


In [6]:
print(missing_dates.head(3))

                                           Headlines        Dates  \
0                            VW Makes Big Bet on EVs  20 Jul 2010   
1  BREAKING: Honda Plans Electric Car, Plug-in Hy...  20 Jul 2010   
2  Glenn Beck Going Blind Macular Degeneration - ...  20 Jul 2010   

                                        Descriptions  
0  Volkswagen is diving into electric vehicles wi...  
1  Just like last week's Toyota-Tesla announcemen...  
2  Conservative talk-show host Glenn Beck recentl...  


In [7]:
def convert_to_datetime(dates):
    list = dates.split(" ")
    day = list[0]
    month = list[1][:3]
    year = list[2]
    date = f"{day} {month} {year}"
    return datetime.strptime(date, "%d %b %Y")

missing_dates['Dates'] = missing_dates['Dates'].apply(convert_to_datetime)

In [8]:
print(missing_dates.head(3))

                                           Headlines      Dates  \
0                            VW Makes Big Bet on EVs 2010-07-20   
1  BREAKING: Honda Plans Electric Car, Plug-in Hy... 2010-07-20   
2  Glenn Beck Going Blind Macular Degeneration - ... 2010-07-20   

                                        Descriptions  
0  Volkswagen is diving into electric vehicles wi...  
1  Just like last week's Toyota-Tesla announcemen...  
2  Conservative talk-show host Glenn Beck recentl...  


In [11]:
from bisect import bisect_left
import pandas as pd
from datetime import datetime

stock_price = pd.read_csv('stock_price.csv')
stock_price['Date'] = pd.to_datetime(stock_price['Date'])
stock_dates = stock_price['Date'].tolist()
def find_nearest_future_date(target, dates):
    """Find the nearest future date in 'dates' to the 'target' date."""
    pos = bisect_left(dates, target)
    if pos == len(dates):
        # Target is later than any date we have, return None or keep original
        return None
    return dates[pos]
missing_dates['MappedDates'] = missing_dates['Dates'].apply(lambda d: find_nearest_future_date(d, stock_dates) if d not in stock_dates else d)

In [12]:
print(missing_dates.head(30))

                                            Headlines      Dates  \
0                             VW Makes Big Bet on EVs 2010-07-20   
1   BREAKING: Honda Plans Electric Car, Plug-in Hy... 2010-07-20   
2   Glenn Beck Going Blind Macular Degeneration - ... 2010-07-20   
3   IndyCar's new chassis: Plenty of questions remain 2010-07-22   
4                  Lincoln Kills the 'Hybrid Premium' 2010-07-22   
5   White Zombie Electric Race Car Does 0 to 60 mp... 2010-07-22   
6   Nvidia chip team gets 25 million dollars from ... 2010-08-10   
7   GM aplies for Patent on Opel Insignia OPC in C... 2010-08-10   
8   Ford F-150 News: 2011 Harley-Davidson F-150 Re... 2010-08-10   
9   IPL: Infiniti spells performance with new cust... 2010-08-11   
10            Long Term Update 2: 2010 Buick LaCrosse 2010-08-12   
11                   Tesla Roadster Confirmed For GT5 2010-08-16   
12                          Lamborghini V10 in the S6 2010-08-16   
13            A first look at the Coda electric 

In [13]:
tesla_old = tesla_old.drop('Dates', axis=1)
missing_dates = missing_dates.drop('Dates', axis=1)
print(tesla_old.head(3))
print(missing_dates.head(3))

                                           Headlines  \
0  Tesla IPO Raises $226.1M, Stock Surges 41 Percent   
1  2010: Tesla IPO: Electric-car maker’s stock up...   
2    Tesla Motors IPO a win, but road ahead is bumpy   

                                        Descriptions MappedDates  
0  Tesla Motors raised $226.1 million dollars in ...  2010-06-29  
1  2010: Tesla IPO: Electric-car maker's stock up...  2010-06-29  
2  The Tesla Motors IPO sold out quickly, and sha...  2010-06-29  
                                           Headlines  \
0                            VW Makes Big Bet on EVs   
1  BREAKING: Honda Plans Electric Car, Plug-in Hy...   
2  Glenn Beck Going Blind Macular Degeneration - ...   

                                        Descriptions MappedDates  
0  Volkswagen is diving into electric vehicles wi...  2010-07-20  
1  Just like last week's Toyota-Tesla announcemen...  2010-07-20  
2  Conservative talk-show host Glenn Beck recentl...  2010-07-20  


In [14]:
combined_df = pd.concat([tesla_old, missing_dates], ignore_index=True)

In [15]:
print(combined_df.head(3))

                                           Headlines  \
0  Tesla IPO Raises $226.1M, Stock Surges 41 Percent   
1  2010: Tesla IPO: Electric-car maker’s stock up...   
2    Tesla Motors IPO a win, but road ahead is bumpy   

                                        Descriptions MappedDates  
0  Tesla Motors raised $226.1 million dollars in ...  2010-06-29  
1  2010: Tesla IPO: Electric-car maker's stock up...  2010-06-29  
2  The Tesla Motors IPO sold out quickly, and sha...  2010-06-29  


In [18]:
combined_df['MappedDates'] = pd.to_datetime(combined_df['MappedDates'])
combined_df.sort_values(by=['MappedDates'], inplace=True)

In [19]:
all_dates = set(stock_price['Date'])
unique_dates = set(combined_df['MappedDates'])
missing_dates = []

for date in all_dates:
    if date not in unique_dates:
        missing_dates.append(date)

missing_dates.sort()

print(missing_dates)

[Timestamp('2010-07-30 00:00:00'), Timestamp('2010-08-13 00:00:00'), Timestamp('2010-09-03 00:00:00'), Timestamp('2010-09-17 00:00:00'), Timestamp('2011-07-15 00:00:00'), Timestamp('2011-12-15 00:00:00'), Timestamp('2014-07-02 00:00:00')]


In [20]:
merged_df = pd.merge(combined_df, stock_price, left_on='MappedDates', right_on='Date', how='left')

In [22]:
merged_df = merged_df[['Headlines', 'Descriptions', 'MappedDates', 'Close']]


In [23]:
print(merged_df.head(3))

                                           Headlines  \
0  Tesla IPO Raises $226.1M, Stock Surges 41 Percent   
1  Voros: Tesla proves Wall Street still believes...   
2  Tesla Ups Price, Volume of Initial Public Offe...   

                                        Descriptions MappedDates     Close  
0  Tesla Motors raised $226.1 million dollars in ...  2010-06-29  1.592667  
1  Fundamentals or the lack of will catch up to T...  2010-06-29  1.592667  
2  Tesla Motors, Inc. (Nasdaq:TSLA), a manufactur...  2010-06-29  1.592667  


In [ ]:
# this will yield tesla_data_no_perc_no_combined.csv